In [5]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from datetime import timedelta
from datetime import datetime
import requests
import zipfile 
import os


Extrair ITR (Formulário de Informações Trimestrais) da CVM e salvar arquivos (1 para cada ano)

In [6]:
path_dados_trimestrais = "C:\\workspace_mestrado\\dados_cvm_trimestrais\\"
#chdir tem a função de mudar o diretório de trabalho
os.chdir(f"{path_dados_trimestrais}")

anos = range(2011, 2025)
url_base = "http://dados.cvm.gov.br//dados//CIA_ABERTA//DOC//ITR//DADOS//"

for ano in anos:
    download = requests.get(url_base + f'itr_cia_aberta_{ano}.zip')
    open(f"itr_cia_aberta_{ano}.zip", "wb").write(download.content)

Junta os arquivos trimestrais de dados em 1 só arquivo

In [7]:
path_dados_trimestrais = "C:\\workspace_mestrado\\dados_cvm_trimestrais\\"

lista_demonstracoes_2010_2022 = []
diretorio_atual = os.getcwd()

for arquivo in os.listdir(f"{path_dados_trimestrais}"):
    if arquivo.endswith(".zip"):
        with zipfile.ZipFile(arquivo, "r") as zipped:
            zipped.extractall()
            print(f"Arquivo {arquivo} extraído")
            for arquivo_csv in zipped.namelist():
                if arquivo_csv.endswith(".csv") and "DRE" in arquivo_csv and "con" in arquivo_csv:
                    #pegar apenas os arquivos que tiverem DRE no nome
                    demonstracao = pd.read_csv(arquivo_csv, sep=";", decimal=",", encoding="ISO-8859-1", dtype= {"ORDEM_EXERC": "category"})
                    lista_demonstracoes_2010_2022.append(demonstracao)
                    
                    print(f"*Arquivo {arquivo_csv} adicionado à lista de demonstrações")
                os.remove(arquivo_csv)



Arquivo itr_cia_aberta_2011.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2011.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2012.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2012.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2013.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2013.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2014.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2014.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2015.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2015.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2016.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2016.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2017.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2017.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2018.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2018.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2019.zip extraído
*Arquivo itr_ci

Elimina dados desnecessários e salva arquivo em .csv

In [8]:
dase_dados_tri = pd.concat(lista_demonstracoes_2010_2022)

dase_dados_tri[['con_ind', 'tipo_dem']] = dase_dados_tri['GRUPO_DFP'].str.split(' - ', expand=True)
dase_dados_tri['tipo_dem'] = dase_dados_tri['tipo_dem'].str.strip()
dase_dados_tri = dase_dados_tri[dase_dados_tri['ORDEM_EXERC'] != 'PENÚLTIMO']

#remoção de colunas que não serão utilizadas
dase_dados_tri = dase_dados_tri.drop(columns=['CNPJ_CIA', 'MOEDA', 'ORDEM_EXERC', 'con_ind', 'GRUPO_DFP', 'tipo_dem'])


dase_dados_tri['DT_REFER'] = pd.to_datetime(dase_dados_tri['DT_REFER'], format='ISO8601')
dase_dados_tri['DT_INI_EXERC'] = pd.to_datetime(dase_dados_tri['DT_INI_EXERC'], format='ISO8601')
dase_dados_tri['DT_FIM_EXERC'] = pd.to_datetime(dase_dados_tri['DT_FIM_EXERC'], format='ISO8601')


dase_dados_tri.to_csv('C:\\workspace_mestrado\\dados_cvm_trimestrais\\base_dados_tri_2024.csv', sep=";")
print("Arquivo salvo com sucesso!")

Arquivo salvo com sucesso!
